In [2]:
import numpy as np
from collections import defaultdict
import pandas as pd
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
import re, sys, os
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D
from IPython.display import display
import seaborn.apionly as sns
plt.style.use('ggplot')
sns.set_style('ticks')
import matplotlib.ticker as ticker
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from torchvision import datasets
from IPython.display import Image
import h5py
import numpy
import theano
import theano.tensor as T
from theano import pp

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.optim.lr_scheduler import ReduceLROnPlateau
# sys.path.insert(0, "/Users/mahzadkhoshlessan/Desktop/Machine-Learning/Intro-to-Deep-Learning/Final_Project/")
# from CNN_Model import project_01

# Enable inline plotting  
%matplotlib inline

In [2]:
sys.path.append("/Users/mahzadkhoshlessan/Desktop/Machine-Learning/Intro-to-Deep-Learning/Final_Project/PyTorch")

In [3]:
from CNN_Model import project_01, normalize_im, NeuralNet, L1L2loss, weight_init, Checkpoint

In [4]:
# For reproducibility

In [5]:
np.random.seed(123)

In [6]:
# Load training data and divide it to training and validation sets

In [14]:
matfile = h5py.File('../Input/TrainingSet.mat', 'r')

In [15]:
list(matfile.keys())

['heatmaps', 'patches', 'spikes']

In [16]:
patches = np.array(matfile['patches'])

In [17]:
# patches = patches[:50,:,:]    #**********

In [18]:
np.shape(patches)

(10000, 208, 208)

In [19]:
heatmaps = 100.0*np.array(matfile['heatmaps'])   # Why multiply by 100

In [20]:
# heatmaps = heatmaps[:50,:,:]  #**********

In [21]:
X_train, X_test, y_train, y_test = train_test_split(patches, heatmaps, test_size=0.3, random_state=42)

In [ ]:
# Setting type
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

In [22]:
np.shape(X_train)

(7000, 208, 208)

In [15]:
print('Number of Training Examples: %d' % X_train.shape[0])
print('Number of Validation Examples: %d' % X_test.shape[0])

Number of Training Examples: 7000
Number of Validation Examples: 3000


In [16]:
#===================== Training set normalization ==========================
# normalize training images to be in the range [0,1] and calculate the 
# training set mean and std

In [17]:
mean_train = np.zeros(X_train.shape[0],dtype=np.float32)
std_train = np.zeros(X_train.shape[0], dtype=np.float32)
for i in range(X_train.shape[0]):
    X_train[i, :, :] = project_01(X_train[i, :, :])
    mean_train[i] = X_train[i, :, :].mean()
    std_train[i] = X_train[i, :, :].std()

In [18]:
# resulting normalized training images
mean_val_train = mean_train.mean()
std_val_train = std_train.mean()
X_train_norm = np.zeros(X_train.shape, dtype=np.float32)
for i in range(X_train.shape[0]):
    X_train_norm[i, :, :] = normalize_im(X_train[i, :, :], mean_val_train, std_val_train)

In [19]:
# patch size (Input Dim)
psize = X_train_norm.shape[1]

In [20]:
# ===================== Test set normalization ==========================
# normalize test images to be in the range [0,1] and calculate the test set 
# mean and std

In [21]:
mean_test = np.zeros(X_test.shape[0],dtype=np.float32)
std_test = np.zeros(X_test.shape[0], dtype=np.float32)
for i in range(X_test.shape[0]):
    X_test[i, :, :] = project_01(X_test[i, :, :])
    mean_test[i] = X_test[i, :, :].mean()
    std_test[i] = X_test[i, :, :].std()

In [22]:
# resulting normalized test images
mean_val_test = mean_test.mean()
std_val_test = std_test.mean()
X_test_norm = np.zeros(X_test.shape, dtype=np.float32)
for i in range(X_test.shape[0]):
    X_test_norm[i, :, :] = normalize_im(X_test[i, :, :], mean_val_test, std_val_test)

In [23]:
P = dict()
P['numEpoch'] = 6
P['learning_rate'] = 0.001
P['batchSize'] = 16

### Reshaping: nn.Conv2d expects an input of the shape [batch_size, channels, height, width].

In [25]:
X_train_norm = X_train_norm.view(X_train.shape[0], 1, psize, psize)
X_test_norm = X_test_norm.view(X_test.shape[0], 1, psize, psize)
y_train = y_train.view(y_train.shape[0], 1, psize, psize)
y_test = y_test.view(y_test.shape[0], 1, psize, psize)

In [26]:
train = data_utils.TensorDataset(X_train_norm, y_train)
test = data_utils.TensorDataset(X_test_norm, y_test)

train_loader = DataLoader(train,shuffle=True,batch_size=P['batchSize'])
test_loader = DataLoader(test,shuffle=False,batch_size=P['batchSize'])

In [27]:
myModel = NeuralNet()

In [ ]:
optimizer = optim.Adam(myModel.parameters(), lr=P['learning_rate'])
criterion = L1L2loss((1, psize, psize))

In [ ]:
myModel.apply(weight_init)
scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5, min_lr=0.00005)

In [ ]:
# Define the loss history recorder

In [ ]:
df = pd.DataFrame(columns=('epoch', 'loss', 'loss_test'))
Weight_file = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'Checkpointing/Checkpointer.h5')))
meanstd_name = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'meanstd_name.mat')))
path = os.path.abspath(os.path.normpath(os.path.join(os.getcwd(),'output.csv')))

In [ ]:
len_dataset = {'train':N_train,'test':N_test}
loss_list = {'train':[],'test':[]}
loss_c = {'train':np.inf,'test':np.inf}
data_loader = {'train':train_loader,'test':test_loader}

In [ ]:
for epoch in range(P['numEpoch']):
    for phase in ['train', 'test']:
        if phase == 'train':
            myModel.train()
        else:
            myModel.eval()
    
        running_loss = 0.0
        counter = 0
        for i, (images, labels) in enumerate(data_loader[phase]):
            # Run the forward pass
            counter = counter+1
            if phase=='train' and counter==400:
                break

            density_pred = myModel(images)
            loss = criterion(labels,density_pred)
        
            if phase == 'train':
                scheduler.optimizer.zero_grad()
                loss.backward()
                scheduler.step(loss_c['test']) 

            running_loss += loss.detach().numpy()

        running_loss *= P['batchSize']/N_train 
        loss_list[phase].append(running_loss) 

        # Save the model weights after each epoch if the validation loss decreased
        loss_c[phase] = running_loss
        if phase=='test':
            if loss_list['test']:
                is_best = bool(loss_c['test'] < min(loss_list['test']))    
                Checkpoint(myModel, optimizer, epoch, is_best, Weight_file) 
    
        loss_list[phase].append(running_loss) 

    # B.3) stat for this epoch
    print(str(epoch)+'    loss:'+str(loss_c['train'])+' '+str(loss_c['test']))
    # save
    df.loc[epoch] = [epoch, loss_c['train'], loss_c['test']]
    df.to_csv(path)   

In [ ]:
mdict = {"mean_test": mean_val_test, "std_test": std_val_test}                  
sio.savemat(meanstd_name, mdict) 

In [ ]:
dff = pd.read_csv('keras-data/Keras_loss.csv', sep=r'\t',engine='python', header=None)

In [ ]:
dff1 = pd.read_csv('keras-data/Keras-loss-no-scheduler.csv', sep=r'\t',engine='python', header=None)

In [ ]:
dff2 = pd.read_csv('keras-data/output111.csv', sep=r',',engine='python')

In [ ]:
dff3 = pd.read_csv('keras-data/output.csv', sep=r',',engine='python')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))

ax.plot(dff[0],dff[1],'-o')
ax.plot(dff[0],dff[2],'-o')

ax.plot(dff1[0],dff1[1],'-o')
ax.plot(dff1[0],dff1[2],'-o')

ax.plot(dff2['epoch'],dff2['loss'],'-o')
ax.plot(dff2['epoch'],dff2['loss_test'],'-o')

# ax.plot(dff3['epoch'],dff3['loss'],'-o')
# ax.plot(dff3['epoch'],dff3['loss_test'],'-o')

ax.grid()
ax.set_xlabel('epochs')
ax.set_ylabel('Loss')

ax.legend(['train loss Keras', 'test loss Keras','train loss Keras-No Scheduler', 'test loss Keras-No Scheduler',
           'train loss Torch', 'test loss Torch'])

fig.savefig('loss_out.pdf')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
ax.plot(dff3['epoch'],dff3['loss'],'-o')
ax.plot(dff3['epoch'],dff3['loss_test'],'-o')

ax.grid()
ax.set_xlabel('epochs')
ax.set_ylabel('Loss')

ax.legend(['train loss Torch-No Scheduler-No Init', 'test loss Torch-No Scheduler-No Init'])
fig.savefig('loss_out2.pdf')

In [ ]:
if cuda:
    checkpoint = torch.load(resume_weights)
else:
    # Load GPU model on CPU
    checkpoint = torch.load(resume_weights,
                            map_location=lambda storage,
                            loc: storage)
start_epoch = checkpoint['epoch']
best_accuracy = checkpoint['best_accuracy']
model.load_state_dict(checkpoint['state_dict'])
print("=> loaded checkpoint '{}' (trained for {} epochs)".format(resume_weights, checkpoint['epoch']))
